In [5]:
import sqlite3

In [39]:
## User input
while True:
    try:
        selection = int(input('''
            1: Luo tietokanta
            2: Lisää uusi paikka
            3: Lisää asiakas
            4: Lisää lähetys
            5: Lisää tapahtuma
            6: Hae kaikki paketin tapahtumat
            7: Hae kaikki asiakkaan paketit ja niihin liittyvien tapahtumien määrä
            8: Hae annetusta paikasta tapahtumien määrä tiettynä päivänä
            9: Tehokkuustesti
        '''))
        if 1 <= selection <= 9:
            print('number is fine')
            break
        else:
            print('VIRHE: Syötä numero väliltä 1-9')
    except NameError:
        print('VIRHE: Syötä mumero väliltä 1-9')

number is fine


In [53]:
## 1. Create database and connect to it
try:
    open('c:/Users/arttu/tikape/sql/tikape.db')
    db = sqlite3.connect('c:/Users/arttu/tikape/sql/tikape.db')
    db.isolation_level = None
    c = db.cursor()
    sql_schema = open('c:/Users/arttu/tikape/sql/1-schema.sql', 'r').read()
    c.executescript(sql_schema)
    print('Tietokanta luotu')
except FileExistsError:
    print('VIRHE: Tietokanta on jo olemassa')

OperationalError: table Asiakas already exists

In [54]:
## 2. Add place
selection = 2
if selection == 2:
    paikka = input('Anna paikan nimi: \n')

    # Check if exists already
    c.execute('''SELECT paikka
                 FROM Paikka
                 WHERE paikka =?
                 ''', [paikka])
    result = c.fetchone()
    if result:
        print('VIRHE: Paikka on jo olemassa')    
    else:
        c.execute('''INSERT INTO Paikka
                     (paikka)
                     VALUES
                     (?);
                     ''', [paikka])
        print('Paikka lisätty')


Paikka lisätty


In [57]:
## 3. Add customer
selection = 3
if selection == 3:
    asiakas = input('Anna asiakkaan nimi: \n')
    c.execute('''SELECT asiakas
                 FROM Asiakas
                 WHERE asiakas =?;
                 ''', [asiakas])
    result = c.fetchone()
    if result:
        print('VIRHE: Asiakas on jo olemassa')
    else:
        c.execute('''INSERT INTO Asiakas
                 (asiakas)
                 VALUES
                 (?);
                 ''', [asiakas])
    print('Asiakas lisätty')

Asiakas lisätty


In [60]:
## 4. Add delivery
selection = 4
if selection == 4:
    paketti = input('Anna paketin seurantakoodi: \n')
    asiakas_id = input('Anna asiakkaan nimi: \n')

    c.execute('''SELECT paketti
                  FROM Paketti
                  WHERE paketti =?
              ''', [paketti])
    result = c.fetchone()
    if result:
        print('VIRHE: Lähetys on jo olemassa')
    else:
        c.execute('''INSERT INTO Paketti
             (paketti, asiakas_id)
             VALUES
             (?, ?);
             ''', [paketti, asiakas_id])
        print('Paketti lisätty')    

Paketti lisätty


In [67]:
# 5. Add event
selection = 5
if selection == 5:
    paketti = input('Anna paketin seurantakoodi: \n')
    paikka = input('Anna tapahtuman paikka: \n')
    kuvaus = input('Anna tapahtuman kuvaus')
    
    # Check if place exists
    c.execute('''SELECT paikka
                 FROM Paikka
                 WHERE paikka =?
              ''', [paikka])
    result = c.fetchone()
    if not result:
        print('VIRHE: Paikkaa ei ole olemassa')
    # Check if package exists
    c.execute('''SELECT paketti
                 FROM Paketti
                 WHERE paketti =?
              ''', [paketti])
    result = c.fetchone()
    if not result:
        print('VIRHE: Lähetyskoodia ei ole olemassa')
    
    else:
        c.execute('''INSERT INTO Tapahtuma
                     (paketti_id, paikka, pvm, kuvaus)
                     VALUES
                     (?, ?, datetime('now'), ?);
                 ''', [seurantakoodi, paikka, kuvaus])
        print('Tapahtuma lisätty')

Tapahtuma lisätty


In [71]:
# 6. Print events
paketti_id = input('Anna paketin lähetystunnus \n')
c.execute('''SELECT *
             FROM Tapahtuma
             WHERE paketti_id =?
         ''', [paketti_id])
print(c.fetchall())

[(2, 'a1', 'naara', '2020-02-27 18:32:10', 'testi')]


In [73]:
# 7. Print packages and counts of events
asiakas = input('Anna asiakkaan nimi \n')
c.execute('''SELECT t.paketti_id, count(t.id)
             FROM Tapahtuma t, Paketti p
             WHERE p.asiakas_id =?
             GROUP BY t.paketti_id
          ''', [asiakas])
print(c.fetchall())

[('', 1), ('a1', 1)]


In [76]:
# Print events per place and date
paikka = input('Anna paikan nimi \n')
pvm = input('Anna päivämäärä (2020-02-28) \n')
c.execute('''SELECT t.paikka_id, count(*)
             FROM Tapahtuma t, Paikka p
             WHERE t.paikka_id = p.paikka AND p.paikka =? AND date(?)
             GROUP BY t.paikka_id
          ''', [paikka, pvm])
print(c.fetchall())

[('naara', 1)]
